In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
from pathlib import Path
import plotly.graph_objects as go
from typing import Iterable

# Fix import
sys.path.append(os.path.abspath(os.path.join('..')))
from facilitators import BarFacilitator, StackedQuantityEvolutionFacilitatorBase
from data import read_sol_keys

In [2]:
PROJECT_ROOT_DIR = Path().resolve().parent
SOLUTION_DIR = PROJECT_ROOT_DIR / "solution_files"

DAILY_DATA = SOLUTION_DIR / "solution_data_daily_d20_h6_s6_no_invest.sol"
SEASON_DATA = SOLUTION_DIR / "solution_data_season_d20_h6_s6_no_invest.sol"

WRITE_DIR = PROJECT_ROOT_DIR / "generated_figs"
WRITE_FIGS = True

REGION = "ALL"
YEAR = 2050

# Stacked Capacity Evolution

In [43]:

def stacked_capacity_evolution_fig(sol_path: str, region: str = "ALL", title_addon: str = None, write_fig = False) -> tuple[go.Figure, Iterable[str]]:
    storage_evolution = StackedQuantityEvolutionFacilitatorBase(sol_path=sol_path, type_of_data_to_read="CapitalInvestment", 
                                                                region=region, extra_identifying_columns=["Technology"])

    d = storage_evolution.get_relevant_data()
    d.filter_by_containing_string(column="Technology", identifier="D_")

    if region == "ALL":
        d.aggreagate_all_by_sum(column_to_aggregate="Region", aggregated_entry_name=region, column_to_sum="Value")

    # Remove those who are 0 for all years
    techs_over_zero = d.df[d.df["Value"] > 0]["Technology"].unique()
    d.filter_by_list(column="Technology", identifier_list=techs_over_zero)

    fig = storage_evolution.plot(storage_evolution.generate_traces(d))

    fig.update_layout(
        width=1400,
        height=800,
        title=f"Storage Capacity, Region: {region}" + f", {title_addon}" if title_addon is not None else "",
        yaxis_title="GW",
        xaxis_title="Year",
    )

    if write_fig:
        fig.write_image(WRITE_DIR / f"stacked_capacity_evolution_{region}_{title_addon}.png")

    return fig, techs_over_zero


In [44]:
stacked_capacity_evolution_fig(DAILY_DATA, region=REGION, title_addon="Daily", write_fig=WRITE_FIGS)[0].show()
fig, techs_over_zero_capacity = stacked_capacity_evolution_fig(SEASON_DATA, region=REGION, title_addon="Seasonal", write_fig=WRITE_FIGS)
fig.show()

No unit conversion applied!


No unit conversion applied!


# Bar Capacity Evolution

In [45]:
def bar_capacity_evolution_fig(tech: str, region: str, daily_data: Path, season_data: Path, write_fig: bool = False) -> go.Figure:

    bar = BarFacilitator(sol_paths={"daily": daily_data, "season": season_data}, type_of_data_to_read="TotalCapacityAnnual", 
                        x_grouping_columns=["Year", "Source"], legend_grouping_columns=["Technology", "Source"])

    data = bar.get_relevant_data()

    #data.filter_by_list(column="Technology", identifier_list=["D_Battery_Li-Ion"])
    data.filter_by_identifier(column="Technology", identifier=tech)

    if region == "ALL":
        data.aggreagate_all_by_sum(column_to_aggregate="Region", aggregated_entry_name=region, column_to_sum="Value")

    data.filter_by_identifier(column="Region", identifier=region)

    fig = bar.plot(bar.generate_traces(data))

    fig.update_layout(
        title="Total Capacity Annual",
        xaxis_title="Year",
        yaxis_title="Capacity (GW)",
        width=1200,
        height=600
        )
    
    if write_fig:
        fig.write_image(WRITE_DIR / f"bar_capacity_evolution_{tech}_{region}.png")
    
    return fig

In [2]:
for tech in techs_over_zero_capacity:
    bar_capacity_evolution_fig(tech, REGION, DAILY_DATA, SEASON_DATA, write_fig=WRITE_FIGS).show()
#bar_capacity_evolution_fig("D_Battery_Li-Ion", REGION, DAILY_DATA, SEASON_DATA, write_fig=WRITE_FIGS).show()

NameError: name 'techs_over_zero_capacity' is not defined

# Hourly Rate Activity

In [3]:
def hourly_rate_of_activity_fig(tech: str, region: str, year: int, daily_data: Path, season_data: Path, write_fig: bool = False) -> go.Figure:

    bar = BarFacilitator(sol_paths={"daily": daily_data, "season": season_data}, type_of_data_to_read="RateOfActivity", 
                        x_grouping_columns=["TS", "Source"], legend_grouping_columns=["Technology", "Source"])

    data = bar.get_relevant_data()

    # Fix yearsplit division
    data.df["Value"] *= data.year_split

    # Keep only tech
    data.filter_by_identifier(column="Technology", identifier=tech)

    # Keep only modes 1 and 2
    data.filter_by_list("Mode", [1, 2])

    # Set mode 1 to negative
    data.df.loc[data.df["Mode"] == 2, "Value"] *= -1

    # Aggregate over Mode
    data.aggreagate_all_by_sum(column_to_aggregate="Mode", aggregated_entry_name=-1, column_to_sum="Value")

    # Get unique timesteps, and get the one hitting daily stuff
    timesteps = data.df["TS"].unique()
    timesteps.sort()
    hourstep = timesteps[1] - timesteps[0]
    start_hour = timesteps[0]

    zero_timesteps = []
    for t in timesteps:
        if (t - hourstep + start_hour) % 24 == 0:
            zero_timesteps.append(t)

    print("Zero timesteps", zero_timesteps)

    if region == "ALL":
        data.aggreagate_all_by_sum(column_to_aggregate="Region", aggregated_entry_name=region, column_to_sum="Value")

    data.df = data.df[data.df["Year"] == year]

    traces = bar.generate_traces(data)

    # Add traces to mark zero timesteps
    #for i, t in enumerate(zero_timesteps):
    #    traces.append(go.Scatter(x=[[t], ["daily"]], y=[0], mode="markers", showlegend = i == 0, name="Daily reset",
    #                             marker=dict(color="red", size=10)))
    traces.append(go.Scatter(x=[zero_timesteps, ["daily"]*len(zero_timesteps)], y=[0]*len(zero_timesteps), 
                             mode="markers", showlegend = True, name="Daily reset",
                             marker=dict(color="red", size=10)))

    fig = bar.plot(traces)

    fig.update_layout(
            title=f"Rate of activity, Region: {region}, Year: {year}",
            yaxis_title="TWh",
            xaxis_title="Year",
            width=1200,
            height=600
        )
    
    if write_fig:
        fig.write_image(WRITE_DIR / f"hourly_rate_of_activity_{tech}_{region}_{year}.png")

    return fig

In [4]:
for tech in techs_over_zero_capacity:
    hourly_rate_of_activity_fig(tech, REGION, YEAR, DAILY_DATA, SEASON_DATA, write_fig=WRITE_FIGS).show()

#hourly_rate_of_activity_fig("D_Battery_Li-Ion", REGION, YEAR, DAILY_DATA, SEASON_DATA, write_fig=WRITE_FIGS).show()

NameError: name 'techs_over_zero_capacity' is not defined

# More